In [1]:
import rtdl
import torch
import torch.nn as nn
import torch.nn.functional as F

In [8]:
# Load the data
import pandas as pd

train_path = '../data_old/train.csv'
dev_in_path = '../data_old/dev_in.csv'

df_train = pd.read_csv(train_path)
df_train.head()


,fact_time,fact_latitude,fact_longitude,fact_temperature,fact_cwsm_class,climate,topography_bathymetry,sun_elevation,climate_temperature,climate_pressure,...,cmc_0_1_66_0_grad,cmc_0_1_66_0_next,cmc_0_1_67_0_grad,cmc_0_1_67_0_next,cmc_0_1_68_0_grad,cmc_0_1_68_0_next,gfs_2m_dewpoint_grad,gfs_2m_dewpoint_next,gfs_total_clouds_cover_low_grad,gfs_total_clouds_cover_low_next
0,1.543321e+09,26.968800,-99.248901,2.0,0.0,dry,127.0,-17.526443,14.613571,754.263405,...,0.0,0.0,0.0,0.0,0.0,0.0,-2.600006,-2.750006,0.0,0.0
1,1.538776e+09,29.374201,-100.927002,31.0,20.0,mild temperate,297.0,41.531032,26.992143,733.117168,...,0.0,0.0,0.0,0.0,0.0,0.0,-0.600006,17.950006,-12.0,11.0
2,1.552115e+09,22.149599,113.592003,17.0,10.0,mild temperate,-1.0,43.916531,18.842143,761.571076,...,0.0,0.0,0.0,0.0,0.0,0.0,-0.233978,21.450006,1.0,8.0
3,1.549566e+09,34.678699,-86.684799,24.0,20.0,mild temperate,193.0,40.240955,8.303571,747.524910,...,0.0,0.0,0.0,0.0,0.0,0.0,0.059448,16.150018,-58.0,41.0
4,1.552910e+09,46.066667,41.966667,9.0,20.0,dry,90.0,30.394660,6.451429,753.168113,...,0.0,0.0,0.0,0.0,0.0,0.0,0.400024,3.150018,18.0,92.0


In [9]:
df_dev_in = pd.read_csv(dev_in_path)
df_dev_in.head()

,fact_time,fact_latitude,fact_longitude,fact_temperature,fact_cwsm_class,climate,topography_bathymetry,sun_elevation,climate_temperature,climate_pressure,...,cmc_0_1_66_0_grad,cmc_0_1_66_0_next,cmc_0_1_67_0_grad,cmc_0_1_67_0_next,cmc_0_1_68_0_grad,cmc_0_1_68_0_next,gfs_2m_dewpoint_grad,gfs_2m_dewpoint_next,gfs_total_clouds_cover_low_grad,gfs_total_clouds_cover_low_next
0,1539162000,-40.350000,-9.880000,11.0,10,tropical,-843.0,31.782490,10.070714,765.631228,...,0.00000,0.0000,0.0,0.000000,0.0,0.0,0.505035,2.647577,2.0,2.0
1,1545006600,53.421299,-6.270070,4.0,10,mild temperate,67.0,-59.691521,7.005000,752.897615,...,0.00000,0.0000,0.0,0.000000,0.0,0.0,-0.400024,1.249994,0.0,0.0
2,1540094400,-19.757700,63.361000,26.0,10,dry,6.0,35.250889,23.327143,763.115016,...,0.00000,0.0000,0.0,0.000000,0.0,0.0,0.100006,21.050012,-1.0,1.0
3,1552611600,35.245899,47.009201,5.0,10,mild temperate,1390.0,-23.755615,3.109286,609.419333,...,1.69672,5.1653,0.0,0.000049,0.0,0.0,-1.500000,-0.349982,-12.0,81.0
4,1545631200,26.633333,118.150000,14.0,20,mild temperate,210.0,33.040438,12.172143,734.678037,...,0.00000,0.0000,0.0,0.000000,0.0,0.0,0.102081,11.513879,-15.0,83.0


In [10]:
dev_out_path = '../data_old/dev_out.csv'
eval_in_path = '../data_old/eval_in.csv'
eval_out_path = '../data_old/eval_out.csv'

df_dev_out = pd.read_csv(dev_out_path)
df_eval_in = pd.read_csv(eval_in_path)
df_eval_out = pd.read_csv(eval_out_path)

In [11]:
# Identify the categorical features
cat_features = []
for col in df_dev_in:
    values = df_dev_in[col].tolist()
    unique = list(dict.fromkeys(values))
    if len(unique) < 20:
        cat_features.append(col)
print(cat_features)

['fact_cwsm_class', 'climate', 'cmc_available', 'gfs_available', 'gfs_soil_temperature_available', 'gfs_timedelta_s', 'wrf_available']


In [12]:
# !pip install scipy
from scipy import stats
import numpy as np

nan_replacements = {}
for col in df_train:
    if col in cat_features:
        nan_replacements[col] = stats.mode(np.asarray(df_train[col].tolist()))[0][0]
        # print(nan_replacements[col])
    else:
        nan_replacements[col] = np.mean(np.asarray(df_train[col].dropna().tolist()))
print(nan_replacements.values())

dict_values([1545219635.2602534, 27.3491997995972, -19.454721235367906, 15.030736293991083, 0.0, 'mild temperate', 309.93658796685, -2.7249874731371273, 15.053651434869764, 731.2756530851661, 289.1659257771938, 0.0513848381221949, 287.04881832634976, 287.0853175734476, 287.0339327353252, 258.9922501121575, 274.97475431188, 282.77351197349384, 285.8909051108904, 281.3276019215079, 7.660611531165116, 5.534873822269684, 14.704511863943937, 12.971739936778564, 8.78866233485156, 7.2775056043593676, 0.009529923266778687, 0.00859610917297107, 6.413124162323838, 0.2673820249969054, 0.027512247376225554, 0.008222988603064724, 3.866966535018241e-05, -0.0020003342605871715, -0.05948973513452053, 10.42531664950694, 5.2929354486362215, 1.8376276146288066, 0.4768480677913728, -0.09701087752828826, -0.14489588205685547, 0.22814633531193007, 0.2219977889627698, 0.30237136633108197, 0.18710502254215602, 97345.58898593165, 97344.2322319218, 101661.80541725569, 137.95607755299883, 5728.521789532874, 3094

In [13]:
# Replace nans
for col in df_train:
    df_train[col] = df_train[col].fillna(nan_replacements[col])
    df_dev_in[col] = df_dev_in[col].fillna(nan_replacements[col])
    df_dev_out[col] = df_dev_out[col].fillna(nan_replacements[col])
    df_eval_in[col] = df_eval_in[col].fillna(nan_replacements[col])
    df_eval_out[col] = df_eval_out[col].fillna(nan_replacements[col])

In [14]:
# !pip install scikit-learn

from typing import Dict

import numpy as np
import sklearn.preprocessing


def normalize(
    X: Dict[str, np.ndarray], normalization: str, seed: int, noise: float = 1e-3
) -> Dict[str, np.ndarray]:
    # X ~ {'train': <train_size x n_features>, 'val': <val_size x n_features>, 'test': <test_size x n_features>}
    X_train = X['train']
    if normalization == 'standard':
        normalizer = sklearn.preprocessing.StandardScaler()
    elif normalization == 'quantile':
        normalizer = sklearn.preprocessing.QuantileTransformer(
            output_distribution='normal',
            n_quantiles=max(min(X['train'].shape[0] // 30, 1000), 10),
            subsample=1e9,
            random_state=seed,
        )
        if noise:
            X_train = X_train.copy()
            stds = np.std(X_train, axis=0, keepdims=True)
            noise_std = noise / np.maximum(stds, noise)
            X_train += noise_std * np.random.default_rng(seed).standard_normal(
                X_train.shape
            )
    else:
        raise ValueError(f'unknown normalization: {normalization}')
    normalizer.fit(X_train)
    return {k: normalizer.transform(v) for k, v in X.items()}

In [15]:
# Set Seed
seed = 100

In [16]:
# Normalize using training stats
# Quantile normalisation is used (maps to a normal distribution)
X_train_np = np.asarray(df_train.iloc[:,6:])
X_dev_in_np = np.asarray(df_dev_in.iloc[:,6:])
X_dev_out_np = np.asarray(df_dev_out.iloc[:,6:])
X_eval_in_np = np.asarray(df_eval_in.iloc[:,6:])
X_eval_out_np = np.asarray(df_eval_out.iloc[:,6:])

In [17]:
X = {'train': X_train_np, 'dev_in': X_dev_in_np, 'dev_out': X_dev_out_np, 'eval_in': X_eval_in_np, 'eval_out': X_eval_out_np}
X = normalize(X, normalization='quantile', seed=100)

In [18]:
df_train.loc[:,6:] = X['train']
df_dev_in.loc[:,6:] = X['dev_in']
df_dev_out.loc[:,6:] = X['dev_out']
df_eval_in.loc[:,6:] = X['eval_in']
df_eval_out.loc[:,6:] = X['eval_out']

/home/miproj/4thyr.oct2019/vr311/venv_tabTrans/lib/python3.7/site-packages/pandas/core/indexing.py:719: FutureWarning: Slicing a positional slice with .loc is not supported, and will raise TypeError in a future version.  Use .loc with labels or .iloc with positions instead.
  indexer = self._get_setitem_indexer(key)


In [19]:
df_train.head()

,fact_time,fact_latitude,fact_longitude,fact_temperature,fact_cwsm_class,climate,topography_bathymetry,sun_elevation,climate_temperature,climate_pressure,...,cmc_0_1_66_0_grad,cmc_0_1_66_0_next,cmc_0_1_67_0_grad,cmc_0_1_67_0_next,cmc_0_1_68_0_grad,cmc_0_1_68_0_next,gfs_2m_dewpoint_grad,gfs_2m_dewpoint_next,gfs_total_clouds_cover_low_grad,gfs_total_clouds_cover_low_next
0,1.543321e+09,26.968800,-99.248901,2.0,0.0,dry,0.109004,-0.332947,-0.023254,0.323360,...,-0.049644,-0.139538,-0.010017,-0.093874,-0.001574,-0.016849,-1.811459,-1.070745,0.024693,-0.568109
1,1.538776e+09,29.374201,-100.927002,31.0,20.0,mild temperate,0.555462,1.134250,1.304557,-0.501024,...,-0.049644,-0.139538,-0.010017,-0.093874,-0.001574,-0.016849,-0.658618,0.811230,-1.253139,0.554625
2,1.552115e+09,22.149599,113.592003,17.0,10.0,mild temperate,-1.597528,1.216301,0.338383,1.132273,...,-0.049644,-0.139538,-0.010017,-0.093874,-0.001574,-0.016849,-0.280781,1.238268,0.863079,0.508006
3,1.549566e+09,34.678699,-86.684799,24.0,20.0,mild temperate,0.325577,1.090354,-0.651507,-0.051622,...,-0.049644,-0.139538,-0.010017,-0.093874,-0.001574,-0.016849,0.084409,0.664750,-2.256744,0.862197
4,1.552910e+09,46.066667,41.966667,9.0,20.0,dry,-0.055435,0.752285,-0.883372,0.242020,...,-0.049644,-0.139538,-0.010017,-0.093874,-0.001574,-0.016849,0.457329,-0.429243,1.415641,1.438232


In [20]:
# Save modified dataframes
out_dir = '../data_preprocessed'
df_train.to_csv(f'{out_dir}/train.csv', index=False)
df_dev_in.to_csv(f'{out_dir}/dev_in.csv', index=False)
df_dev_out.to_csv(f'{out_dir}/dev_out.csv', index=False)
df_eval_in.to_csv(f'{out_dir}/eval_in.csv', index=False)
df_eval_out.to_csv(f'{out_dir}/eval_out.csv', index=False)